📘 1️⃣ Project Introduction

# 🎓 AI-Powered Higher Education Chatbot (UniBot)

This notebook builds a fully functional AI chatbot for universities:

- Upload documents: PDF, CSV, Excel, TXT  
- Automatic text extraction + OCR for scanned PDFs  
- Vector indexing using Sentence Transformers  
- Retrieval-Augmented Generation (RAG)  
- Mistral-7B Instruct model (4-bit quantized for GPU efficiency)  
- Flask web application with modern UI  
- Persistent chat history  
- Multi-document search  

This notebook installs dependencies, prepares the vector database, 
removes sensitive tokens, and launches the web interface.


🟩  Install Dependencies

In [1]:
# ===============================
# 1️⃣ Install dependencies
# ===============================

!pip install flask pyngrok transformers accelerate bitsandbytes
!pip install chromadb sentence-transformers langchain-community langchain pypdf

!mkdir -p templates static uploads vectorstore


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━

In [2]:
!pip install langchain-text-splitters

In [3]:
# System deps (run in Colab)
!apt update -y
!apt install -y poppler-utils tesseract-ocr

# Python deps
!pip install flask transformers bitsandbytes sentence-transformers langchain-community chromadb pypdf pytesseract pdf2image


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,470 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,149 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu 

📘 3️⃣ Hugging Face Login (Token Removed)

## 🔐 Hugging Face Authentication (User Must Add Token)

Your notebook previously contained a **hardcoded token**, which is insecure.

It is now removed.

### To use gated models on Hugging Face:

1. Go to: https://huggingface.co/settings/tokens  
2. Click **New Token**  
3. Select **Read** access  
4. Copy your token  
5. Paste it in the code below:



In [ ]:
# ===============================
# 2️⃣ Hugging Face Authentication
# ===============================
from huggingface_hub import login
login(token="YOUR_TOKEN_HERE")   # replace with your token


📘 4️⃣ Create the Flask Backend (app.py)

This cell writes the complete Flask application that powers the AI chatbot.

Includes:

- Mistral-7B LLM loading (4-bit GPU)
- Document ingestion (PDF, CSV, TXT, Excel)
- OCR fallback for scanned PDFs
- Vector store storage (ChromaDB)
- Semantic search + retrieval
- Chat history persistence for each session
- API routes for upload, chat, history, listing files

The logic remains exactly the same as your original project — only cleaned,
token-safe, and ready for use on Kaggle.


In [ ]:
# ===============================
# Write app.py
# ===============================

%%writefile app.py
import os
import io
import uuid
import json
import functools
import traceback
import torch
from datetime import datetime
from flask import (
    Flask, render_template, request, jsonify, send_from_directory,
    make_response
)

# Transformers + quantization
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
)

# Embeddings + vectorstore
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# PDF reading + OCR
from pypdf import PdfReader
import pytesseract
from pdf2image import convert_from_path

# CSV + Excel support
import csv
import pandas as pd


# ===============================
# FLASK SETUP
# ===============================
app = Flask(__name__)
app.config["UPLOAD_FOLDER"] = "uploads"
app.config["CHAT_HISTORY_FOLDER"] = "chat_histories"

os.makedirs(app.config["UPLOAD_FOLDER"], exist_ok=True)
os.makedirs(app.config["CHAT_HISTORY_FOLDER"], exist_ok=True)
os.makedirs("vectorstore", exist_ok=True)

session_histories = {}


# ===============================
# GLOBAL CONFIG
# ===============================
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
EMBED_MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
RETRIEVE_K = 5


# ===============================
# Load LLM (4-bit GPU)
# ===============================
@functools.lru_cache(maxsize=1)
def load_llm():
    print("🚀 Loading Mistral-7B on GPU...")

    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float16,
        quantization_config=bnb
    )

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.2,
    )

    print("✔ LLM Loaded.")
    return pipe


# ===============================
# Embedding Model
# ===============================
@functools.lru_cache(maxsize=1)
def load_embedder():
    print("Loading embedding model...")
    return HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME)


# ===============================
# Vectorstore Loader
# ===============================
def get_vectorstore():
    return Chroma(
        collection_name="university_docs",
        embedding_function=load_embedder(),
        persist_directory="vectorstore"
    )


# ===============================
# PDF Processing (with OCR fallback)
# ===============================
def process_pdf(file_path):
    text = ""

    try:
        reader = PdfReader(file_path)
        for page in reader.pages:
            try:
                extracted = page.extract_text()
            except:
                extracted = None
            if extracted and extracted.strip():
                text += extracted + "\n"
    except:
        pass

    warning = None
    if not text.strip():
        warning = "📸 OCR used — scanned PDF detected"
        try:
            images = convert_from_path(file_path, dpi=200)
            for img in images:
                try:
                    text += pytesseract.image_to_string(img) + "\n"
                except:
                    pass
        except:
            pass

    if not text.strip():
        return 0, "❌ No extractable text found"

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=60)
    chunks = splitter.split_text(text)
    docs = [Document(page_content=c) for c in chunks]

    vs = get_vectorstore()
    vs.add_documents(docs)

    return len(chunks), warning


# ===============================
# CSV / TXT / Excel Processing
# ===============================
def process_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        text = "CSV Data:\n" + df.to_string()

        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=60)
        chunks = splitter.split_text(text)
        docs = [Document(page_content=c) for c in chunks]

        vs = get_vectorstore()
        vs.add_documents(docs)
        return len(chunks), None

    except Exception as e:
        return 0, f"❌ CSV processing error: {str(e)}"


def process_txt(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()

        if not text.strip():
            return 0, "❌ No text found in file"

        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=60)
        chunks = splitter.split_text(text)
        docs = [Document(page_content=c) for c in chunks]

        vs = get_vectorstore()
        vs.add_documents(docs)
        return len(chunks), None

    except Exception as e:
        return 0, f"❌ Text processing error: {str(e)}"


def process_xlsx(file_path):
    try:
        df = pd.read_excel(file_path)
        text = "Excel Data:\n" + df.to_string()

        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=60)
        chunks = splitter.split_text(text)
        docs = [Document(page_content=c) for c in chunks]

        vs = get_vectorstore()
        vs.add_documents(docs)
        return len(chunks), None

    except Exception as e:
        return 0, f"❌ Excel processing error: {str(e)}"


# ===============================
# Retrieval Function
# ===============================
def retrieve_context(query):
    vs = get_vectorstore()
    try:
        docs = vs.similarity_search(query, k=RETRIEVE_K)
        return "\n\n".join(d.page_content for d in docs)
    except:
        return ""


# ===============================
# Chat History Helpers
# ===============================
def save_chat_history(sid):
    if sid in session_histories:
        try:
            path = os.path.join(app.config["CHAT_HISTORY_FOLDER"], f"{sid}.json")
            with open(path, "w") as f:
                json.dump(session_histories[sid], f)
        except:
            pass


def load_chat_history(sid):
    try:
        path = os.path.join(app.config["CHAT_HISTORY_FOLDER"], f"{sid}.json")
        if os.path.exists(path):
            with open(path, "r") as f:
                session_histories[sid] = json.load(f)
        else:
            session_histories[sid] = []
    except:
        session_histories[sid] = []


# ===============================
# Answer Generation
# ===============================
def generate_answer(query, sid):
    history = ""
    if sid in session_histories:
        last = session_histories[sid][-6:]
        history = "\n".join(f"{m['role']}: {m['text']}" for m in last)

    context = retrieve_context(query)
    if not context.strip():
        context = "No relevant content in uploaded documents."

    prompt = f"""
You are UniBot — an intelligent university assistant AI.
Help students with academic queries, university policies, admissions, syllabi, and general information.
Always cite document references when using uploaded content.
Be concise, helpful, and professional.

Recent conversation:
{history}

Knowledge base:
{context}

Student: {query}

Answer:
"""

    llm = load_llm()
    output = llm(prompt)[0]["generated_text"]

    if "Answer:" in output:
        return output.split("Answer:")[-1].strip()

    return output.replace(prompt, "").strip()


# ===============================
# Session ID Helper
# ===============================
def get_or_create_session_id(request):
    sid = request.cookies.get("session_id")
    if not sid:
        sid = str(uuid.uuid4())
        load_chat_history(sid)
    else:
        if sid not in session_histories:
            load_chat_history(sid)
    return sid


# ===============================
# ROUTES
# ===============================
@app.route("/")
def home():
    resp = make_response(render_template("index.html"))
    if "session_id" not in request.cookies:
        sid = str(uuid.uuid4())
        load_chat_history(sid)
        resp.set_cookie("session_id", sid, max_age=2592000)
    return resp


@app.route("/chat")
def chat():
    sid = get_or_create_session_id(request)
    files = [
        f for f in os.listdir(app.config["UPLOAD_FOLDER"])
        if f.split(".")[-1].lower() in ["pdf", "csv", "txt", "xlsx", "xls"]
    ]
    resp = make_response(render_template("chat.html", uploaded_files=files))
    resp.set_cookie("session_id", sid, max_age=2592000)
    return resp


@app.route("/uploads/<path:filename>")
def serve_upload(filename):
    return send_from_directory(app.config["UPLOAD_FOLDER"], filename)


@app.route("/list_files")
def list_files():
    files = [
        f for f in os.listdir(app.config["UPLOAD_FOLDER"])
        if f.split(".")[-1].lower() in ["pdf", "csv", "txt", "xlsx", "xls"]
    ]
    return jsonify({"success": True, "files": files})


@app.route("/upload", methods=["POST"])
def upload_pdf():
    try:
        if "file" not in request.files:
            return jsonify({"success": False, "error": "No file uploaded"}), 400

        f = request.files["file"]
        orig = f.filename
        ext = orig.split(".")[-1].lower()
        uid = f"{uuid.uuid4()}.{ext}"
        path = os.path.join(app.config["UPLOAD_FOLDER"], uid)
        f.save(path)

        if ext == "pdf":
            chunks, warning = process_pdf(path)
        elif ext == "csv":
            chunks, warning = process_csv(path)
        elif ext == "txt":
            chunks, warning = process_txt(path)
        elif ext in ["xlsx", "xls"]:
            chunks, warning = process_xlsx(path)
        else:
            return jsonify({"success": False, "error": "Unsupported file type"}), 400

        return jsonify({
            "success": True,
            "original_filename": orig,
            "stored_filename": uid,
            "chunks": chunks,
            "warning": warning
        })

    except Exception as e:
        traceback.print_exc()
        return jsonify({"success": False, "error": str(e)}), 500


@app.route("/ask", methods=["POST"])
def ask():
    try:
        data = request.get_json(force=True, silent=True)
        if not data:
            return jsonify({"success": False, "error": "Invalid JSON"}), 400

        query = (data.get("query") or "").strip()
        if not query:
            return jsonify({"success": False, "error": "Empty query"}), 400

        sid = get_or_create_session_id(request)
        session_histories[sid].append({
            "role": "user",
            "text": query,
            "timestamp": datetime.now().isoformat()
        })

        try:
            answer = generate_answer(query, sid)
        except:
            traceback.print_exc()
            return jsonify({"success": False, "error": "LLM generation failed"}), 500

        session_histories[sid].append({
            "role": "assistant",
            "text": answer,
            "timestamp": datetime.now().isoformat()
        })

        save_chat_history(sid)

        return jsonify({"success": True, "answer": answer})

    except Exception as e:
        traceback.print_exc()
        return jsonify({"success": False, "error": str(e)}), 500


@app.route("/get_history")
def get_history():
    sid = get_or_create_session_id(request)
    return jsonify({"success": True, "history": session_histories.get(sid, [])})


@app.route("/clear_history", methods=["POST"])
def clear_history():
    sid = get_or_create_session_id(request)
    session_histories[sid] = []
    save_chat_history(sid)
    return jsonify({"success": True})


# ===============================
# RUN SERVER
# ===============================
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)


Writing app.py


📘 5️⃣ Create Frontend Template — index.html
This file defines the home page of UniBot.

It includes:
- Navbar
- Hero section
- Features section
- Theme toggle (dark/light)
- Navigation to Chat page
- Modern UI with glassmorphism

Logic is unchanged. Only the token references are removed
and formatting is preserved exactly as your original version.


In [ ]:
# ===============================
# Write templates/index.html
# ===============================

%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en" data-theme="dark">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>🎓 UniBot - AI University Assistant</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>

<body class="home-body">
  <header class="navbar glass-morphism">
    <div class="navbar-container">
      <div class="logo-section">
        <div class="logo-icon">🎓</div>
        <div class="logo-text">
          <h1>UniBot</h1>
          <p>AI Assistant</p>
        </div>
      </div>

      <nav class="nav-links">
        <a href="/" class="nav-link active">Home</a>
        <a href="/chat" class="nav-link cta-button">💬 Start Chat</a>
        <button class="theme-toggle" onclick="toggleTheme()" title="Toggle theme">
          <span class="theme-icon">🌙</span>
        </button>
      </nav>
    </div>
  </header>

  <main class="hero-section">
    <div class="hero-container">
      <div class="hero-content glass-morphism">
        <div class="hero-text">
          <h2 class="hero-title">
            <span class="gradient-text">AI-Powered</span><br>
            University Assistant
          </h2>
          <p class="hero-subtitle">
            Upload syllabus, handbooks, PDFs, CSVs, and more. Ask anything about your university documents.
            Powered by advanced AI that understands context.
          </p>

          <div class="features-grid">
            <div class="feature-card">
              <span class="feature-icon">📄</span>
              <span class="feature-title">Multi-Format</span>
              <p>PDF, CSV, Excel, TXT</p>
            </div>
            <div class="feature-card">
              <span class="feature-icon">🤖</span>
              <span class="feature-title">Smart AI</span>
              <p>Mistral 7B LLM</p>
            </div>
            <div class="feature-card">
              <span class="feature-icon">💾</span>
              <span class="feature-title">Persistent</span>
              <p>Chat History Saved</p>
            </div>
            <div class="feature-card">
              <span class="feature-icon">⚡</span>
              <span class="feature-title">Fast</span>
              <p>GPU Accelerated</p>
            </div>
          </div>

          <div class="cta-group">
            <a href="/chat" class="btn btn-primary btn-lg">
              <span class="btn-icon">🚀</span> Start Chat Now
            </a>
            <a href="#features" class="btn btn-secondary btn-lg">
              <span class="btn-icon">📚</span> Learn More
            </a>
          </div>
        </div>

        <div class="hero-visual">
          <div class="floating-card card-1">
            <div class="card-icon">📊</div>
            <p>Analyze Data</p>
          </div>
          <div class="floating-card card-2">
            <div class="card-icon">🎯</div>
            <p>Get Answers</p>
          </div>
          <div class="floating-card card-3">
            <div class="card-icon">💡</div>
            <p>Learn Fast</p>
          </div>
        </div>
      </div>
    </div>

    <section id="features" class="features-section">
      <div class="section-container">
        <h3 class="section-title">Why Choose UniBot?</h3>

        <div class="benefits-grid">
          <div class="benefit glass-morphism">
            <div class="benefit-icon">🔍</div>
            <h4>Smart Search</h4>
            <p>Semantic search through all your uploaded documents instantly</p>
          </div>
          <div class="benefit glass-morphism">
            <div class="benefit-icon">🧠</div>
            <h4>Contextual Answers</h4>
            <p>AI understands context and provides detailed, accurate responses</p>
          </div>
          <div class="benefit glass-morphism">
            <div class="benefit-icon">📱</div>
            <h4>Responsive Design</h4>
            <p>Works perfectly on desktop, tablet, and mobile devices</p>
          </div>
          <div class="benefit glass-morphism">
            <div class="benefit-icon">🔐</div>
            <h4>Private & Secure</h4>
            <p>Your documents and chat history stay private in your session</p>
          </div>
        </div>
      </div>
    </section>
  </main>

  <footer class="footer">
    <p>Made with ❤️ • UniBot © 2024 • AI University Assistant</p>
  </footer>

  <script>
    function toggleTheme() {
      const html = document.documentElement;
      const current = html.getAttribute("data-theme");
      const newTheme = current === "dark" ? "light" : "dark";
      html.setAttribute("data-theme", newTheme);
      localStorage.setItem("theme", newTheme);
    }

    // Load saved theme
    const savedTheme = localStorage.getItem("theme") || "dark";
    document.documentElement.setAttribute("data-theme", savedTheme);
  </script>
</body>
</html>


Writing templates/index.html


📘 6️⃣ Create Chat Interface — chat.html

This file defines the main chat interface of UniBot.

Features included:
- Document upload (PDF, CSV, TXT, Excel)
- Indexed files sidebar
- Chat window with message bubbles
- Typing indicator
- Dark/light theme toggle
- Toast notifications
- Smooth animations and responsive layout

The UI is modern, fast, and mobile-friendly.


In [ ]:
# ===============================
# Write templates/chat.html
# ===============================

%%writefile templates/chat.html
<!DOCTYPE html>
<html lang="en" data-theme="dark">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>UniBot Chat</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>

<body class="chat-body">
  <header class="navbar glass-morphism">
    <div class="navbar-container">
      <div class="logo-section">
        <div class="logo-icon">🎓</div>
        <div class="logo-text">
          <h1>UniBot</h1>
          <p>Chat</p>
        </div>
      </div>

      <nav class="nav-links">
        <a href="/" class="nav-link">Home</a>
        <a href="/chat" class="nav-link active">Chat</a>
        <button class="theme-toggle" onclick="toggleTheme()" title="Toggle theme">
          <span class="theme-icon">🌙</span>
        </button>
      </nav>
    </div>
  </header>

  <main class="chat-main">
    <!-- SIDEBAR -->
    <aside class="sidebar glass-morphism">
      <div class="sidebar-header">
        <h2>📚 Documents</h2>
        <button class="btn btn-icon" onclick="refreshFiles()" title="Refresh files">🔄</button>
      </div>

      <!-- FILE UPLOAD -->
      <div class="upload-section">
        <label class="upload-dropzone" id="uploadDropzone">
          <input type="file" id="fileInput" accept=".pdf,.csv,.txt,.xlsx,.xls"
                 multiple style="display:none">

          <div class="upload-content">
            <div class="upload-icon">📤</div>
            <div class="upload-text">
              <strong>Drop files here</strong>
              <small>or click to browse</small>
            </div>
            <div class="upload-hint">📄 PDF, CSV, TXT, Excel</div>
          </div>
        </label>

        <button id="uploadBtn" class="btn btn-primary btn-full" onclick="uploadFile()">
          ⬆️ Upload & Index
        </button>
      </div>

      <!-- UPLOADED FILE LIST -->
      <div class="files-container">
        <h3 class="files-title">✅ Indexed Files</h3>
        <div id="filesList" class="files-list">
          {% for f in uploaded_files %}
            <a href="{{ url_for('serve_upload', filename=f) }}" target="_blank"
               class="file-item glass-morphism">
              <span class="file-icon">
                {% if f.endswith('.pdf') %}📄{% elif f.endswith('.csv') %}
                📊{% elif f.endswith('.xlsx') or f.endswith('.xls') %}
                📈{% else %}📋{% endif %}
              </span>
              <span class="file-name" title="{{ f }}">{{ f[:20] }}...</span>
            </a>
          {% endfor %}
        </div>
      </div>

      <div class="sidebar-footer">
        <button class="btn btn-danger btn-full" onclick="clearAllHistory()">
          🗑️ Clear All
        </button>
      </div>
    </aside>

    <!-- MAIN CHAT -->
    <section class="chat-container">
      <div id="chatMessages" class="chat-messages">
        <div class="chat-welcome">
          <div class="welcome-icon">🎓</div>
          <h3>Welcome to UniBot!</h3>
          <p>Upload your documents and start asking questions</p>
          <div class="welcome-hints">
            <div class="hint">💡 Ask about syllabus content</div>
            <div class="hint">📊 Analyze data from CSVs</div>
            <div class="hint">❓ Get instant answers</div>
          </div>
        </div>
      </div>

      <!-- INPUT AREA -->
      <div class="chat-input-area glass-morphism">
        <div class="input-wrapper">
          <textarea
            id="userInput"
            class="message-input"
            placeholder="Ask a question about your documents... (Press Ctrl+Enter to send)"
            maxlength="2000"></textarea>

          <div class="char-counter">
            <span id="charCount">0</span>/2000
          </div>
        </div>

        <div class="input-controls">
          <button id="sendBtn" class="btn btn-primary btn-send" onclick="sendMessage()">
            📤 Send
          </button>
        </div>

        <div id="statusBar" class="status-bar">
          <span class="status-indicator idle">●</span>
          <span class="status-text">Ready</span>
        </div>
      </div>
    </section>
  </main>

  <!-- TOASTS -->
  <div id="toastContainer" class="toast-container"></div>

  <script src="{{ url_for('static', filename='script.js') }}"></script>

  <script>
    function toggleTheme(){
      const html = document.documentElement;
      const current = html.getAttribute("data-theme");
      const newTheme = current === "dark" ? "light" : "dark";
      html.setAttribute("data-theme", newTheme);
      localStorage.setItem("theme", newTheme);
    }

    const savedTheme = localStorage.getItem("theme") || "dark";
    document.documentElement.setAttribute("data-theme", savedTheme);

    loadChatHistory();
    refreshFiles();
  </script>
</body>
</html>


Writing templates/chat.html


📘 7️⃣ Create Stylesheet — static/style.css

This file defines all visual styling for UniBot:

- Full dark/light theme system  
- Responsive layout  
- Glassmorphism effects  
- Hero section, navbar, cards  
- Chat UI design (messages, bubbles, sidebar)  
- Buttons, loader, animations  
- Scrollbars, toast notifications  

The CSS is unchanged from your original project.


In [ ]:
# ===============================
# Write static/style.css
# ===============================

%%writefile static/style.css
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700;800&family=JetBrains+Mono:wght@400;600&display=swap');

/* ===========================
   COLOR THEME VARIABLES
   =========================== */
html[data-theme="dark"] {
  --bg-primary: #0a0e27;
  --bg-secondary: #1a1f3a;
  --bg-tertiary: #252d4a;
  --text-primary: #ffffff;
  --text-secondary: #a0aec0;
  --text-tertiary: #718096;
  --border-color: rgba(255, 255, 255, 0.1);
  --glass-bg: rgba(255, 255, 255, 0.05);
  --glass-border: rgba(255, 255, 255, 0.1);
  --accent-primary: #60a5fa;
  --accent-secondary: #3b82f6;
  --accent-tertiary: #8b5cf6;
  --success: #10b981;
  --warning: #f59e0b;
  --error: #ef4444;
  --gradient-1: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
  --gradient-2: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
  --gradient-3: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
}

html[data-theme="light"] {
  --bg-primary: #f8f9fb;
  --bg-secondary: #ffffff;
  --bg-tertiary: #f3f4f6;
  --text-primary: #1f2937;
  --text-secondary: #6b7280;
  --text-tertiary: #9ca3af;
  --border-color: rgba(0, 0, 0, 0.08);
  --glass-bg: rgba(255, 255, 255, 0.7);
  --glass-border: rgba(0, 0, 0, 0.1);
  --accent-primary: #2563eb;
  --accent-secondary: #1d4ed8;
  --accent-tertiary: #7c3aed;
  --success: #059669;
  --warning: #d97706;
  --error: #dc2626;
  --gradient-1: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
  --gradient-2: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
  --gradient-3: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
}

/* ===========================
   GLOBAL STYLES
   =========================== */
* {
  margin: 0;
  padding: 0;
  box-sizing: border-box;
  font-family: 'Inter', sans-serif;
}

html {
  scroll-behavior: smooth;
}

body {
  background-color: var(--bg-primary);
  color: var(--text-primary);
  line-height: 1.6;
  transition: background-color 0.3s, color 0.3s;
}

/* ===========================
   GLASS MORPHISM EFFECT
   =========================== */
.glass-morphism {
  background: var(--glass-bg);
  border: 1px solid var(--glass-border);
  backdrop-filter: blur(20px);
  border-radius: 16px;
  animation: fadeInUp 0.5s ease-out;
}

@keyframes fadeInUp {
  from {
    opacity: 0;
    transform: translateY(20px);
  }
  to {
    opacity: 1;
    transform: translateY(0);
  }
}

.gradient-text {
  background: var(--gradient-1);
  -webkit-background-clip: text;
  -webkit-text-fill-color: transparent;
  background-clip: text;
}

/* ===========================
   NAVBAR
   =========================== */
.navbar {
  position: sticky;
  top: 0;
  z-index: 100;
  padding: 12px 24px;
  margin: 12px;
  margin-bottom: 0;
  border-radius: 20px;
}

.navbar-container {
  max-width: 1400px;
  margin: 0 auto;
  display: flex;
  justify-content: space-between;
  align-items: center;
  gap: 30px;
}

.logo-section {
  display: flex;
  align-items: center;
  gap: 12px;
  cursor: pointer;
  transition: transform 0.3s;
}

.logo-section:hover {
  transform: scale(1.05);
}

.logo-icon {
  font-size: 28px;
  animation: bounce 2s infinite;
}

@keyframes bounce {
  0%, 100% { transform: translateY(0); }
  50% { transform: translateY(-5px); }
}

.logo-text h1 {
  font-size: 20px;
  font-weight: 700;
  margin: 0;
}

.logo-text p {
  font-size: 12px;
  color: var(--text-secondary);
  margin: 0;
}

.nav-links {
  display: flex;
  align-items: center;
  gap: 20px;
}

.nav-link {
  text-decoration: none;
  color: var(--text-secondary);
  font-weight: 500;
  padding: 8px 16px;
  border-radius: 8px;
  transition: all 0.3s;
  position: relative;
}

.nav-link:hover {
  color: var(--text-primary);
  background: rgba(96, 165, 250, 0.1);
}

.nav-link.active {
  color: var(--accent-primary);
  background: rgba(96, 165, 250, 0.15);
}

.nav-link.cta-button {
  background: var(--gradient-1);
  color: white;
  border: none;
}

.theme-toggle {
  background: none;
  border: none;
  font-size: 20px;
  cursor: pointer;
  opacity: 0.7;
  transition: opacity 0.3s, transform 0.3s;
  padding: 4px 8px;
}

.theme-toggle:hover {
  opacity: 1;
  transform: rotate(20deg);
}

/* ===========================
   HOME PAGE - HERO SECTION
   =========================== */
.home-body {
  background: linear-gradient(135deg, var(--bg-primary) 0%, var(--bg-secondary) 100%);
}

.hero-section {
  min-height: calc(100vh - 100px);
  padding: 60px 20px;
}

.hero-container {
  max-width: 1400px;
  margin: 0 auto;
}

.hero-content {
  display: grid;
  grid-template-columns: 1fr 1fr;
  gap: 60px;
  align-items: center;
  padding: 60px;
  min-height: 600px;
}

.hero-text {
  animation: slideInLeft 0.8s ease-out;
}

@keyframes slideInLeft {
  from {
    opacity: 0;
    transform: translateX(-40px);
  }
  to {
    opacity: 1;
    transform: translateX(0);
  }
}

.hero-title {
  font-size: 48px;
  font-weight: 800;
  line-height: 1.2;
  margin-bottom: 20px;
}

.hero-subtitle {
  font-size: 18px;
  color: var(--text-secondary);
  line-height: 1.8;
  margin-bottom: 30px;
}

.features-grid {
  display: grid;
  grid-template-columns: repeat(2, 1fr);
  gap: 16px;
  margin-bottom: 40px;
}

.feature-card {
  display: flex;
  flex-direction: column;
  align-items: center;
  gap: 10px;
  padding: 20px;
  background: rgba(96, 165, 250, 0.1);
  border-radius: 12px;
  text-align: center;
  transition: all 0.3s;
}

.feature-card:hover {
  background: rgba(96, 165, 250, 0.2);
  transform: translateY(-5px);
}

.feature-icon {
  font-size: 32px;
}

.feature-title {
  font-weight: 600;
  color: var(--text-primary);
}

.feature-card p {
  font-size: 14px;
  color: var(--text-secondary);
  margin: 0;
}

.cta-group {
  display: flex;
  gap: 16px;
}

.hero-visual {
  position: relative;
  display: flex;
  align-items: center;
  justify-content: center;
  min-height: 400px;
  animation: slideInRight 0.8s ease-out;
}

@keyframes slideInRight {
  from {
    opacity: 0;
    transform: translateX(40px);
  }
  to {
    opacity: 1;
    transform: translateX(0);
  }
}

.floating-card {
  position: absolute;
  padding: 24px;
  background: var(--glass-bg);
  border: 1px solid var(--glass-border);
  border-radius: 12px;
  display: flex;
  flex-direction: column;
  align-items: center;
  gap: 12px;
  animation: float 6s ease-in-out infinite;
}

@keyframes float {
  0%, 100% { transform: translateY(0px); }
  50% { transform: translateY(-20px); }
}

.card-1 { animation-delay: 0s; top: 20px; left: 20px; }
.card-2 { animation-delay: 1s; bottom: 60px; left: 40px; }
.card-3 { animation-delay: 2s; top: 40px; right: 30px; }

.card-icon {
  font-size: 40px;
}

/* ===========================
   FEATURES SECTION
   =========================== */
.features-section {
  padding: 80px 20px;
  background: linear-gradient(135deg, rgba(96, 165, 250, 0.05), rgba(139, 92, 246, 0.05));
}

.section-container {
  max-width: 1400px;
  margin: 0 auto;
}

.section-title {
  font-size: 36px;
  font-weight: 700;
  text-align: center;
  margin-bottom: 60px;
}

.benefits-grid {
  display: grid;
  grid-template-columns: repeat(auto-fit, minmax(280px, 1fr));
  gap: 24px;
}

.benefit {
  padding: 32px;
  text-align: center;
  transition: all 0.3s;
  cursor: pointer;
}

.benefit:hover {
  transform: translateY(-10px);
  border-color: var(--accent-primary);
}

.benefit-icon {
  font-size: 48px;
  margin-bottom: 16px;
  display: block;
}

.benefit h4 {
  font-size: 20px;
  margin-bottom: 12px;
  color: var(--text-primary);
}

.benefit p {
  color: var(--text-secondary);
  font-size: 14px;
  line-height: 1.6;
}

/* ===========================
   BUTTONS
   =========================== */
.btn {
  padding: 12px 24px;
  border: none;
  border-radius: 10px;
  font-weight: 600;
  font-size: 14px;
  cursor: pointer;
  transition: all 0.3s;
  display: inline-flex;
  align-items: center;
  gap: 8px;
  text-decoration: none;
  white-space: nowrap;
}

.btn-primary {
  background: var(--gradient-1);
  color: white;
}

.btn-primary:hover {
  transform: translateY(-2px);
  box-shadow: 0 8px 24px rgba(96, 165, 250, 0.3);
}

.btn-secondary {
  background: transparent;
  border: 2px solid var(--accent-primary);
  color: var(--accent-primary);
}

.btn-secondary:hover {
  background: rgba(96, 165, 250, 0.1);
  transform: translateY(-2px);
}

.btn-lg {
  padding: 16px 32px;
  font-size: 16px;
}

.btn-full {
  width: 100%;
  justify-content: center;
}

.btn-icon {
  width: 32px;
  height: 32px;
  border-radius: 8px;
  display: flex;
  align-items: center;
  justify-content: center;
  background: rgba(255, 255, 255, 0.1);
}

.btn-danger {
  background: rgba(239, 68, 68, 0.2);
  color: var(--error);
  border: 1px solid var(--error);
}

.btn-danger:hover {
  background: rgba(239, 68, 68, 0.3);
}

/* ===========================
   FOOTER
   =========================== */
.footer {
  padding: 30px 20px;
  text-align: center;
  color: var(--text-secondary);
  border-top: 1px solid var(--border-color);
  font-size: 14px;
}

/* ===========================
   CHAT PAGE LAYOUT
   =========================== */
.chat-body {
  display: flex;
  flex-direction: column;
  height: 100vh;
  overflow: hidden;
}

.chat-main {
  display: grid;
  grid-template-columns: 350px 1fr;
  gap: 16px;
  flex: 1;
  padding: 16px;
  overflow: hidden;
}

.sidebar {
  display: flex;
  flex-direction: column;
  padding: 20px;
  min-height: 0;
  overflow-y: auto;
}

.sidebar-header {
  display: flex;
  justify-content: space-between;
  align-items: center;
  margin-bottom: 20px;
}

.sidebar-header h2 {
  font-size: 18px;
  margin: 0;
}

.upload-section {
  margin-bottom: 24px;
}

.upload-dropzone {
  display: block;
  padding: 20px;
  border: 2px dashed var(--border-color);
  border-radius: 12px;
  text-align: center;
  cursor: pointer;
  transition: all 0.3s;
  margin-bottom: 12px;
}

.upload-dropzone:hover {
  border-color: var(--accent-primary);
  background: rgba(96, 165, 250, 0.1);
}

.upload-content {
  display: flex;
  flex-direction: column;
  align-items: center;
  gap: 10px;
}

.upload-icon {
  font-size: 32px;
}

.upload-text strong {
  display: block;
  color: var(--text-primary);
}

.upload-text small {
  color: var(--text-secondary);
  font-size: 12px;
}

.upload-hint {
  font-size: 12px;
  color: var(--text-tertiary);
  margin-top: 8px;
}

.files-container {
  flex: 1;
  min-height: 0;
  overflow-y: auto;
  margin-bottom: 16px;
}

.files-title {
  font-size: 14px;
  font-weight: 600;
  color: var(--text-secondary);
  margin-bottom: 12px;
  text-transform: uppercase;
  letter-spacing: 0.5px;
}

.files-list {
  display: flex;
  flex-direction: column;
  gap: 8px;
}

.file-item {
  display: flex;
  align-items: center;
  gap: 12px;
  padding: 12px;
  border-radius: 10px;
  text-decoration: none;
  color: var(--text-secondary);
  transition: all 0.3s;
  font-size: 13px;
  cursor: pointer;
}

.file-item:hover {
  background: rgba(96, 165, 250, 0.2);
  color: var(--accent-primary);
  transform: translateX(4px);
}

.file-icon {
  font-size: 20px;
  flex-shrink: 0;
}

.file-name {
  overflow: hidden;
  text-overflow: ellipsis;
  white-space: nowrap;
  flex: 1;
}

/* ===========================
   CHAT CONTAINER
   =========================== */
.chat-container {
  display: flex;
  flex-direction: column;
  gap: 12px;
  background: var(--glass-bg);
  border: 1px solid var(--glass-border);
  border-radius: 16px;
  padding: 20px;
  min-height: 0;
  overflow: hidden;
}

.chat-messages {
  flex: 1;
  overflow-y: auto;
  display: flex;
  flex-direction: column;
  gap: 16px;
  padding-right: 8px;
}

.chat-messages::-webkit-scrollbar {
  width: 6px;
}

.chat-messages::-webkit-scrollbar-track {
  background: transparent;
}

.chat-messages::-webkit-scrollbar-thumb {
  background: var(--accent-primary);
  border-radius: 3px;
}

.chat-welcome {
  display: flex;
  flex-direction: column;
  align-items: center;
  justify-content: center;
  min-height: 300px;
  text-align: center;
  color: var(--text-secondary);
}

.welcome-icon {
  font-size: 64px;
  margin-bottom: 20px;
  animation: bounce 2s infinite;
}

.chat-welcome h3 {
  font-size: 24px;
  color: var(--text-primary);
  margin-bottom: 12px;
}

.chat-welcome p {
  font-size: 16px;
  margin-bottom: 30px;
}

.welcome-hints {
  display: grid;
  grid-template-columns: repeat(3, 1fr);
  gap: 12px;
  max-width: 600px;
}

.hint {
  padding: 12px;
  background: rgba(96, 165, 250, 0.1);
  border-radius: 8px;
  border-left: 3px solid var(--accent-primary);
  font-size: 13px;
}

/* ===========================
   MESSAGE BUBBLES
   =========================== */
.message {
  display: flex;
  gap: 12px;
  animation: slideInUp 0.3s ease-out;
}

@keyframes slideInUp {
  from {
    opacity: 0;
    transform: translateY(20px);
  }
  to {
    opacity: 1;
    transform: translateY(0);
  }
}

.message.user {
  flex-direction: row-reverse;
}

.message-avatar {
  width: 36px;
  height: 36px;
  border-radius: 50%;
  display: flex;
  align-items: center;
  justify-content: center;
  font-size: 20px;
  flex-shrink: 0;
}

.message.assistant .message-avatar {
  background: var(--accent-primary);
  color: white;
}

.message.user .message-avatar {
  background: var(--accent-tertiary);
  color: white;
}

.message-content {
  max-width: 70%;
  padding: 12px 16px;
  border-radius: 14px;
  line-height: 1.5;
  font-size: 14px;
  word-wrap: break-word;
}

.message.assistant .message-content {
  background: var(--glass-bg);
  border: 1px solid var(--glass-border);
  color: var(--text-primary);
}

.message.user .message-content {
  background: var(--accent-primary);
  color: white;
}

.message-time {
  font-size: 12px;
  color: var(--text-tertiary);
  margin-top: 4px;
  text-align: right;
}

.message.user .message-time {
  text-align: left;
}

.typing-indicator {
  display: flex;
  gap: 4px;
  padding: 12px 16px;
}

.typing-dot {
  width: 8px;
  height: 8px;
  border-radius: 50%;
  background: var(--accent-primary);
  animation: typingBounce 1.4s infinite;
}

.typing-dot:nth-child(2) {
  animation-delay: 0.2s;
}

.typing-dot:nth-child(3) {
  animation-delay: 0.4s;
}

@keyframes typingBounce {
  0%, 60%, 100% {
    transform: translateY(0);
    opacity: 0.7;
  }
  30% {
    transform: translateY(-10px);
    opacity: 1;
  }
}

/* ===========================
   INPUT AREA
   =========================== */
.chat-input-area {
  display: flex;
  flex-direction: column;
  gap: 12px;
  padding: 16px;
  border-radius: 12px;
}

.input-wrapper {
  position: relative;
}

.message-input {
  width: 100%;
  min-height: 60px;
  max-height: 200px;
  padding: 14px;
  padding-right: 60px;
  background: var(--bg-tertiary);
  border: 1px solid var(--border-color);
  border-radius: 10px;
  color: var(--text-primary);
  font-family: 'Inter', sans-serif;
  font-size: 14px;
  resize: vertical;
  transition: all 0.3s;
}

.message-input:focus {
  outline: none;
  border-color: var(--accent-primary);
  background: var(--bg-secondary);
  box-shadow: 0 0 0 3px rgba(96, 165, 250, 0.1);
}

.char-counter {
  position: absolute;
  bottom: 10px;
  right: 12px;
  font-size: 12px;
  color: var(--text-tertiary);
}

.input-controls {
  display: flex;
  gap: 12px;
  align-items: center;
}

.btn-send {
  padding: 10px 20px;
  display: flex;
  align-items: center;
  gap: 8px;
}

.send-icon {
  font-size: 16px;
}

.send-text {
  font-weight: 600;
}

.status-bar {
  display: flex;
  align-items: center;
  gap: 8px;
  font-size: 13px;
  color: var(--text-secondary);
  padding: 0 4px;
}

.status-indicator {
  width: 8px;
  height: 8px;
  border-radius: 50%;
  display: inline-block;
  animation: pulse 2s infinite;
}

.status-indicator.idle {
  background: var(--success);
}

.status-indicator.thinking {
  background: var(--accent-primary);
}

.status-indicator.error {
  background: var(--error);
}

@keyframes pulse {
  0%, 100% {
    opacity: 1;
  }
  50% {
    opacity: 0.5;
  }
}

/* ===========================
   TOAST NOTIFICATIONS
   =========================== */
.toast-container {
  position: fixed;
  bottom: 20px;
  right: 20px;
  z-index: 1000;
  display: flex;
  flex-direction: column;
  gap: 12px;
}

.toast {
  background: var(--bg-secondary);
  border: 1px solid var(--border-color);
  padding: 16px 20px;
  border-radius: 10px;
  box-shadow: 0 8px 24px rgba(0, 0, 0, 0.15);
  display: flex;
  align-items: center;
  gap: 12px;
  font-size: 14px;
  animation: slideInRight 0.3s ease-out, slideOutRight 0.3s ease-out 2.7s forwards;
  min-width: 300px;
}

@keyframes slideInRight {
  from {
    opacity: 0;
    transform: translateX(400px);
  }
  to {
    opacity: 1;
    transform: translateX(0);
  }
}

@keyframes slideOutRight {
  from {
    opacity: 1;
    transform: translateX(0);
  }
  to {
    opacity: 0;
    transform: translateX(400px);
  }
}

.toast-icon {
  font-size: 20px;
  flex-shrink: 0;
}

.toast.success {
  border-color: var(--success);
  background: rgba(16, 185, 129, 0.1);
}

.toast.error {
  border-color: var(--error);
  background: rgba(239, 68, 68, 0.1);
}

.toast.warning {
  border-color: var(--warning);
  background: rgba(245, 158, 11, 0.1);
}

/* ===========================
   RESPONSIVE DESIGN
   =========================== */
@media (max-width: 1200px) {
  .chat-main {
    grid-template-columns: 300px 1fr;
  }

  .hero-content {
    grid-template-columns: 1fr;
    gap: 40px;
  }

  .hero-visual {
    min-height: 300px;
  }

  .features-grid {
    grid-template-columns: 1fr;
  }
}

@media (max-width: 768px) {
  .chat-main {
    grid-template-columns: 1fr;
    grid-template-rows: auto 1fr;
  }

  .sidebar {
    grid-column: 1;
    grid-row: 2;
    max-height: 300px;
  }

  .chat-container {
    grid-column: 1;
    grid-row: 1;
    max-height: calc(100vh - 400px);
  }

  .navbar {
    margin: 8px;
  }

  .navbar-container {
    gap: 16px;
  }

  .nav-links {
    gap: 12px;
  }

  .logo-text h1 {
    font-size: 16px;
  }

  .hero-title {
    font-size: 32px;
  }

  .hero-subtitle {
    font-size: 16px;
  }

  .cta-group {
    flex-direction: column;
  }

  .benefits-grid {
    grid-template-columns: 1fr;
  }

  .message-content {
    max-width: 90%;
  }

  .welcome-hints {
    grid-template-columns: 1fr;
  }

  .section-title {
    font-size: 28px;
  }
}

@media (max-width: 480px) {
  .chat-main {
    padding: 8px;
    gap: 8px;
  }

  .sidebar {
    padding: 12px;
  }

  .chat-container {
    padding: 12px;
  }

  .message-content {
    max-width: 100%;
    font-size: 13px;
  }

  .hero-title {
    font-size: 24px;
  }

  .btn-lg {
    padding: 12px 24px;
    font-size: 14px;
  }

  .navbar {
    margin: 6px;
  }

  .logo-text h1 {
    font-size: 14px;
  }

  .toast {
    min-width: 280px;
    right: 10px;
    left: 10px;
  }
}

/* SCROLLBAR */
::-webkit-scrollbar {
  width: 8px;
  height: 8px;
}

::-webkit-scrollbar-track {
  background: transparent;
}

::-webkit-scrollbar-thumb {
  background: var(--accent-primary);
  border-radius: 4px;
}

::-webkit-scrollbar-thumb:hover {
  background: var(--accent-secondary);
}


Writing static/style.css


# 📘 8️⃣ Frontend Script (static/script.js)

The frontend script controls the interaction flow of UniBot.  
It handles:

- Toast notifications  
- Drag-and-drop file uploads  
- Manual file uploads  
- Showing upload progress  
- Refreshing the indexed document list  
- Sending chat messages  
- Displaying user and assistant messages  
- Typing indicator animation  
- Loading previous chat history  
- Clearing chat history  
- Character count for message box  
- Error handling on the frontend  

This file is automatically loaded when the Chat page opens.

**Location:**  
static/script.js


In [9]:

%%writefile static/script.js
// ===========================
// TOAST NOTIFICATION SYSTEM
// ===========================
function showToast(message, type = 'info', duration = 3000) {
  const container = document.getElementById('toastContainer');

  const icons = {
    success: '✅',
    error: '❌',
    warning: '⚠️',
    info: 'ℹ️'
  };

  const toast = document.createElement('div');
  toast.className = `toast ${type}`;
  toast.innerHTML = `
    <span class="toast-icon">${icons[type]}</span>
    <span>${message}</span>
  `;

  container.appendChild(toast);

  setTimeout(() => {
    toast.remove();
  }, duration);
}

// ===========================
// FILE UPLOAD HANDLING
// ===========================
const uploadDropzone = document.getElementById('uploadDropzone');
const fileInput = document.getElementById('fileInput');

if (uploadDropzone && fileInput) {
  uploadDropzone.addEventListener('click', () => fileInput.click());

  uploadDropzone.addEventListener('dragover', (e) => {
    e.preventDefault();
    uploadDropzone.style.borderColor = 'var(--accent-primary)';
    uploadDropzone.style.background = 'rgba(96, 165, 250, 0.1)';
  });

  uploadDropzone.addEventListener('dragleave', () => {
    uploadDropzone.style.borderColor = 'var(--border-color)';
    uploadDropzone.style.background = 'transparent';
  });

  uploadDropzone.addEventListener('drop', (e) => {
    e.preventDefault();
    uploadDropzone.style.borderColor = 'var(--border-color)';
    uploadDropzone.style.background = 'transparent';
    fileInput.files = e.dataTransfer.files;
  });
}

// ===========================
// UPLOAD FILE
// ===========================
async function uploadFile() {
  const fileInput = document.getElementById('fileInput');
  const files = fileInput.files;

  if (!files || files.length === 0) {
    showToast('Please select a file to upload', 'warning');
    return;
  }

  const uploadBtn = document.getElementById('uploadBtn');
  const originalText = uploadBtn.innerHTML;
  uploadBtn.disabled = true;
  uploadBtn.innerHTML = '<span>⏳ Uploading...</span>';

  for (let file of files) {
    const fd = new FormData();
    fd.append('file', file);

    try {
      const res = await fetch('/upload', { method: 'POST', body: fd });
      const data = await res.json();

      if (data.success) {
        showToast(`✅ ${data.original_filename} indexed (${data.chunks} chunks)`, 'success');
        addFileToList(data.original_filename, data.stored_filename);
      } else {
        showToast(`Error: ${data.error}`, 'error');
      }
    } catch (err) {
      showToast(`Upload failed: ${err.message}`, 'error');
    }
  }

  fileInput.value = '';
  uploadBtn.disabled = false;
  uploadBtn.innerHTML = originalText;
}

// ===========================
// ADD FILE TO LIST
// ===========================
function addFileToList(originalName, storedName) {
  const filesList = document.getElementById('filesList');

  const getIcon = (filename) => {
    const ext = filename.split('.').pop().toLowerCase();
    const icons = {
      pdf: '📄',
      csv: '📊',
      txt: '📋',
      xlsx: '📈',
      xls: '📈'
    };
    return icons[ext] || '📄';
  };

  const li = document.createElement('a');
  li.href = `/uploads/${storedName}`;
  li.target = '_blank';
  li.className = 'file-item glass-morphism';
  li.innerHTML = `
    <span class="file-icon">${getIcon(storedName)}</span>
    <span class="file-name" title="${originalName}">${originalName}</span>
  `;

  filesList.prepend(li);
}

// ===========================
// REFRESH FILES
// ===========================
async function refreshFiles() {
  try {
    const res = await fetch('/list_files');
    const data = await res.json();

    if (data.success) {
      const filesList = document.getElementById('filesList');
      filesList.innerHTML = '';

      const getIcon = (filename) => {
        const ext = filename.split('.').pop().toLowerCase();
        const icons = {
          pdf: '📄',
          csv: '📊',
          txt: '📋',
          xlsx: '📈',
          xls: '📈'
        };
        return icons[ext] || '📄';
      };

      data.files.forEach(f => {
        const li = document.createElement('a');
        li.href = `/uploads/${f}`;
        li.target = '_blank';
        li.className = 'file-item glass-morphism';
        li.innerHTML = `
          <span class="file-icon">${getIcon(f)}</span>
          <span class="file-name" title="${f}">${f.substring(0, 30)}...</span>
        `;
        filesList.appendChild(li);
      });
    }
  } catch (err) {
    console.error('Error refreshing files:', err);
  }
}

// ===========================
// CHAT FUNCTIONALITY
// ===========================
const chatMessages = document.getElementById('chatMessages');
const userInput = document.getElementById('userInput');
const sendBtn = document.getElementById('sendBtn');
const statusBar = document.getElementById('statusBar');
const charCount = document.getElementById('charCount');

if (userInput) {
  userInput.addEventListener('input', () => {
    charCount.textContent = userInput.value.length;
  });

  userInput.addEventListener('keypress', (e) => {
    if (e.key === 'Enter' && (e.ctrlKey || e.metaKey)) {
      sendMessage();
    }
  });
}

if (sendBtn) {
  sendBtn.addEventListener('click', sendMessage);
}

async function sendMessage() {
  const message = userInput.value.trim();

  if (!message) {
    showToast('Type a message first', 'warning');
    return;
  }

  if (message.length > 2000) {
    showToast('Message too long (max 2000 characters)', 'error');
    return;
  }

  // Add user message to chat
  addMessageToChat('user', message);
  userInput.value = '';
  charCount.textContent = '0';
  userInput.focus();

  // Update status
  updateStatus('thinking', 'Thinking...');
  sendBtn.disabled = true;

  // Show typing indicator
  const typingId = addTypingIndicator();

  try {
    const res = await fetch('/ask', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({ query: message })
    });

    const data = await res.json();

    removeElement(typingId);

    if (data.success) {
      addMessageToChat('assistant', data.answer);
      updateStatus('idle', 'Ready');
      showToast('Answer received', 'success');
    } else {
      updateStatus('error', 'Error');
      showToast(`Error: ${data.error}`, 'error');
    }
  } catch (err) {
    removeElement(typingId);
    updateStatus('error', 'Connection Error');
    showToast(`Request failed: ${err.message}`, 'error');
  } finally {
    sendBtn.disabled = false;
  }
}

// ===========================
// MESSAGE DISPLAY
// ===========================
function addMessageToChat(role, content) {
  const wrapper = document.createElement('div');
  wrapper.className = `message ${role}`;

  const now = new Date();
  const time = now.toLocaleTimeString([], { hour: '2-digit', minute: '2-digit' });

  wrapper.innerHTML = `
    <div class="message-avatar">${role === 'user' ? '👤' : '🤖'}</div>
    <div>
      <div class="message-content">${escapeHtml(content)}</div>
      <div class="message-time">${time}</div>
    </div>
  `;

  chatMessages.appendChild(wrapper);
  scrollToBottom();

  return wrapper;
}

function addTypingIndicator() {
  const wrapper = document.createElement('div');
  wrapper.className = 'message assistant';
  wrapper.id = 'typing-indicator';
  wrapper.innerHTML = `
    <div class="message-avatar">🤖</div>
    <div class="typing-indicator">
      <div class="typing-dot"></div>
      <div class="typing-dot"></div>
      <div class="typing-dot"></div>
    </div>
  `;

  chatMessages.appendChild(wrapper);
  scrollToBottom();

  return wrapper.id;
}

function removeElement(id) {
  const el = document.getElementById(id);
  if (el) el.remove();
}

function scrollToBottom() {
  setTimeout(() => {
    chatMessages.scrollTop = chatMessages.scrollHeight;
  }, 0);
}

function updateStatus(indicator, text) {
  const statusIndicator = statusBar.querySelector('.status-indicator');
  const statusText = statusBar.querySelector('.status-text');

  statusIndicator.className = `status-indicator ${indicator}`;
  statusText.textContent = text;
}

function escapeHtml(text) {
  const div = document.createElement('div');
  div.textContent = text;
  return div.innerHTML;
}

// ===========================
// LOAD CHAT HISTORY
// ===========================
async function loadChatHistory() {
  try {
    const res = await fetch('/get_history');
    const data = await res.json();

    if (data.success && data.history.length > 0) {
      chatMessages.innerHTML = '';
      data.history.forEach(msg => {
        addMessageToChat(msg.role, msg.text);
      });
    }
  } catch (err) {
    console.error('Error loading chat history:', err);
  }
}

// ===========================
// CLEAR CHAT
// ===========================
async function clearAllHistory() {
  if (!confirm('⚠️ Clear all chat history? This cannot be undone.')) {
    return;
  }

  try {
    const res = await fetch('/clear_history', { method: 'POST' });
    const data = await res.json();

    if (data.success) {
      chatMessages.innerHTML = `
        <div class="chat-welcome">
          <div class="welcome-icon">🎓</div>
          <h3>Welcome to UniBot!</h3>
          <p>Upload your documents and start asking questions</p>
          <div class="welcome-hints">
            <div class="hint">💡 Ask about syllabus content</div>
            <div class="hint">📊 Analyze data from CSVs</div>
            <div class="hint">❓ Get instant answers</div>
          </div>
        </div>
      `;
      showToast('Chat history cleared 🧹', 'success');
    }
  } catch (err) {
    showToast('Error clearing history', 'error');
  }
}

Writing static/script.js


# 📘 9️⃣ Ngrok Setup (User Token Required)

Ngrok is used to expose the Flask application over the internet.

To use Ngrok, users must create their own authentication token.

### Steps to generate a free Ngrok token:
1. Go to: https://dashboard.ngrok.com/signup  
2. Create an account  
3. Visit: https://dashboard.ngrok.com/get-started/your-authtoken  
4. Copy your authentication token

### Paste your token in the notebook:
conf.get_default().auth_token = "YOUR_NGROK_TOKEN_HERE"

Every user should replace this string with their own token.
Do NOT share your token publicly.


# 📘 🔟 Flask + Ngrok Startup Explanation

This cell performs the following tasks:

1. Stops any previous Flask servers  
2. Stops any previous Ngrok tunnels  
3. Shows if port 8000 is already in use  
4. Allows the user to kill the blocking PID  
5. Starts Flask using nohup (runs in background)  
6. Starts a new Ngrok public URL  
7. Displays backend logs from flask.log  

Running this cell ensures that the server starts cleanly
without any conflicts or leftover processes.


# 📘 1️⃣1 Killing Previous Flask & Ngrok Sessions

If Flask or Ngrok is already running in the background,
new tunnels or servers may fail.

Use these commands to stop them:

### Kill all Flask processes:
!pkill -f flask || echo "No flask running"

### Kill all Ngrok processes:
!pkill -f ngrok || echo "No ngrok running"


📘 1️⃣2️⃣ Checking Which Process Uses Port 8000



Flask runs on port 8000.  
To check which process is blocking it:

!lsof -i :8000

Example output:
python3  1758  user   LISTEN  8000

Here, **1758** is the PID currently occupying the port.


In [40]:
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

^C
^C


# 📘 1️⃣3️⃣ Killing a Specific PID

If port 8000 is blocked, kill the process using this:

!kill -9 PID_HERE

Example:
!kill -9 1758

This frees port 8000 for the Flask server.


In [41]:
!lsof -i :8000

In [42]:
!kill -9 1758

/bin/bash: line 1: kill: (1758) - No such process


# 📘 1️⃣4️⃣ Start Flask Backend

Start the Flask server in background mode:

!nohup python app.py > flask.log 2>&1 &

This writes all output logs into flask.log.


In [13]:
!nohup python app.py > flask.log 2>&1 &


In [ ]:
from pyngrok import ngrok, conf
conf.get_default().auth_token = "replace with your ngrok token"  # 🔑 replace with your ngrok token

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

!sleep 3 && tail -n 30 flask.log


🌍 Public URL: NgrokTunnel: "https://6f87d2f99f8c.ngrok-free.app" -> "http://localhost:8000"


In [36]:
!tail -n 50 flask.log

2025-11-20 07:19:48.779018: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763623188.798271    1758 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763623188.804216    1758 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763623188.819160    1758 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763623188.819187    1758 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763623188.819189    1758 computation_placer.cc:177] computation placer alr

In [ ]:
# 📘 1️⃣16 View Flask Logs

# Show the latest 30 lines of logs:

# !tail -n 30 flask.log

# Show more:

# !tail -n 50 flask.log

# These logs are helpful for debugging backend issues.
